In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from PIL import Image
import os

# Function to load images and convert to numpy arrays
def load_images(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            with Image.open(img_path) as img:
                img = img.resize(img_size)
                img_array = np.array(img)
                if img_array.shape == (64, 64, 3):
                    images.append(img_array)
                    labels.append(label)
        except:
            continue
    return np.array(images), np.array(labels)

# Load the dataset
cat_images, cat_labels = load_images('/content/train/cats', 0)
dog_images, dog_labels = load_images('/content/train/dogs', 1)

X = np.concatenate((cat_images, dog_images), axis=0)
y = np.concatenate((cat_labels, dog_labels), axis=0)

# Normalize the images
X = X / 255.0

# Flatten the images
X = X.reshape(X.shape[0], -1)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Reduce dimensionality with PCA
pca = PCA(n_components=150)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [6]:
# Train SVM classifier
svm = SVC(kernel='linear', C=1)
svm.fit(X_train_pca, y_train)

SVC(C=1, kernel='linear')

In [7]:
# Predict on the test set
y_pred = svm.predict(X_test_pca)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 57.85%


In [11]:
import joblib
joblib.dump(svm, 'svm_model.pkl')

['svm_model.pkl']